In [2]:
from pathlib import Path
import json
from datetime import datetime
import re

from tqdm import tqdm
json_dump_db_file = Path('../data/dumps/fact_checks_images_30k.json')
json_dump_db_file.exists()

True

In [3]:
import rich
json_dump_db = json.load(open(json_dump_db_file))

In [4]:
json_dump_db['hits']['total'], len(json_dump_db['hits']['hits'])

({'value': 116296, 'relation': 'eq'}, 30000)

In [5]:
example_record = json_dump_db['hits']['hits'][0]
example_record

{'_index': 'fact_checks',
 '_id': 'PT/OsahOxrKsZGO38sGNKzCfIcU=',
 '_score': 13.485861,
 '_ignored': ['np3k_summary.keyword', 'gpt3_summary.keyword'],
 '_source': {'url': 'https://leadstories.com/hoax-alert/2022/07/fact-check-nasa-did-not-represent-devon-island-canada-as-mars.html',
  'claim': {'default': 'NASA used images of Devon Island to fake photos from Mars'},
  'doc': {'default': 'Analog Mission Did NASA use the landscape of a remote Arctic island in Canada to produce fake images that were then falsely represented as exploration on the planet Mars? No, that\'s not true: This accusation lacks a key element -- there is no evidence that NASA has misrepresented images from the Haughton Mars Project (HMP) or any other analog mission conducted on Earth as if they were images of Mars. NASA\'s website has a page describing analog missions as "field tests in locations that have physical similarities to the extreme space environments." It is true that the terrain of Devon Island, Canada, 

In [6]:
article = example_record['_source']['url']

In [7]:
import sys 
sys.path.append('../')
from src.web_scraper import WebScraper
articles = [article]
scraper = WebScraper(articles)

await scraper.fetch_img_sources()
await scraper.fetch_img_contents()

print(scraper.articles_images)
print(scraper.image_links_files)

{'https://leadstories.com/hoax-alert/2022/07/fact-check-nasa-did-not-represent-devon-island-canada-as-mars.html': ['https://leadstories.com/assets_c/2022/07/screenshot_3479233-thumb-352xauto-3093025.jpg', 'https://static.xx.fbcdn.net/images/emoji.php/v9/te8/1/16/1f451.png', 'https://static.xx.fbcdn.net/images/emoji.php/v9/tfb/1/16/1f5e3.png', 'https://static.xx.fbcdn.net/images/emoji.php/v9/t9e/1/16/1f3b5.png', 'https://static.xx.fbcdn.net/images/emoji.php/v9/t1f/1/16/1f3b6.png', 'https://static.xx.fbcdn.net/images/emoji.php/v9/t49/1/16/1f441.png', 'https://static.xx.fbcdn.net/images/emoji.php/v9/tc8/1/16/1f440.png', 'https://leadstories.com/assets_c/2022/07/liedtopost-thumb-700xauto-3093032.jpg', 'https://leadstories.com/assets_c/2022/07/threeframes-thumb-900x375-3093040.jpg', 'https://leadstories.com/assets_c/2022/07/mastcamcompare-thumb-900x503-3093112.jpg', 'https://leadstories.com/assets_c/2022/07/irelandcompare-thumb-900x429-3093120.jpg', 'https://leadstories.com/assets_c/2022/07

In [8]:
clean_db = {}
for x in tqdm(json_dump_db['hits']['hits']):
    _id = x['_source']['url']
    clean_db[_id] = {
        'url': x['_source']['url'],
        'claim': x['_source']['claim']['default'],
        'content': x['_source']['doc']['default'],
        'top_image': x['_source']['np3k_top_image'],
    }

100%|██████████| 30000/30000 [00:00<00:00, 515530.43it/s]


In [9]:
!cat db_schema.py

from typing import List, Optional
from pydantic import BaseModel

class ESFactModelV2(BaseModel):
    _id: str
    url: str
    claim: str
    content: str
    top_image: str

class ESFactModel(BaseModel):
    claim: str
    label: str
    label4level: str
    doc: str
    url: str
    domain: str
    domain_name: str
    publish_date: str
    keywords: list[str]
    authors: list
    search_engine: str
    lang: str
    score: float
    exact_match: bool




In [18]:
k, v = list(clean_db.items())[0]
k, v

('https://leadstories.com/hoax-alert/2022/07/fact-check-nasa-did-not-represent-devon-island-canada-as-mars.html',
 {'url': 'https://leadstories.com/hoax-alert/2022/07/fact-check-nasa-did-not-represent-devon-island-canada-as-mars.html',
  'claim': 'NASA used images of Devon Island to fake photos from Mars',
  'content': 'Analog Mission Did NASA use the landscape of a remote Arctic island in Canada to produce fake images that were then falsely represented as exploration on the planet Mars? No, that\'s not true: This accusation lacks a key element -- there is no evidence that NASA has misrepresented images from the Haughton Mars Project (HMP) or any other analog mission conducted on Earth as if they were images of Mars. NASA\'s website has a page describing analog missions as "field tests in locations that have physical similarities to the extreme space environments." It is true that the terrain of Devon Island, Canada, does resemble that of Mars -- that\'s why this uninhabited and rocky 

In [19]:
len(clean_db)

28852

In [20]:
# convert to dataframe
import pandas as pd
df = pd.DataFrame.from_dict(clean_db, orient='index')
df['index'] = df.index
df = df.reset_index(drop=True)
df.head()

,url,claim,content,top_image,index
0,https://leadstories.com/hoax-alert/2022/07/fac...,NASA used images of Devon Island to fake photo...,Analog Mission Did NASA use the landscape of a...,https://leadstories.com/caption_3479233.jpg,https://leadstories.com/hoax-alert/2022/07/fac...
1,https://www.boomlive.in/fact-check/no-this-is-...,Photo shows image of death certificate with PM...,An image of a provisional certificate of COVID...,https://www.boomlive.in/h-upload/2021/04/22/94...,https://www.boomlive.in/fact-check/no-this-is-...
2,https://factly.in/the-moroccan-child-in-this-i...,Photo collage featuring the childhood and curr...,A collage containing the images of a girl chil...,https://factly.in/wp-content/uploads//2022/10/...,https://factly.in/the-moroccan-child-in-this-i...
3,https://newsmobile.in/articles/2022/01/28/fact...,Image shows a real photo of Sai Baba,A photograph of a garlanded man is being widel...,https://newsmobile.in/wp-content/uploads/2022/...,https://newsmobile.in/articles/2022/01/28/fact...
4,https://www.vishvasnews.com/english/politics/f...,Image of Kanhaiya Kumar Campaigning behind Afz...,An image is doing rounds on social media in wh...,https://www.vishvasnews.com/wp-content/uploads...,https://www.vishvasnews.com/english/politics/f...


In [21]:
df.to_csv('../data/fact_30k.csv', index=False)